In [ ]:
using JuMP
#using GLPKMathProgInterface
using CPLEX

# Modelo simple

\begin{align}
\text{minimize} \qquad & x+y \\
 \text{subject to} \quad \quad & x+y \geq 1 \\
 \qquad \qquad & x \geq 0, y \geq 0 \\
 \qquad \qquad & x,y \in \mathbb{R}
\end{align}



In [ ]:
#m = Model(solver = GLPKSolverMIP())
#m = Model(solver = GLPKSolverLP())

#creamos un modelo
m = Model(solver = CplexSolver())

#y sus variables
@variable(m, x >= 0 )
@variable(m, y >= 0 )

#función objetivo a minimizar
@objective(m, Min, x + y )

#restricción
@constraint(m, x + y >= 1.0 )

#escribimos el modelo
print(m)

#resolvemos
status = solve(m)

#mostramos resultados
println("Status: ",status," Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

# Un programa lineal en formato estándar

Un PL en formato estándar se expresa como:

$$
\begin{align}
& \text{min} && c^T x \\
& \text{subject to} && A x = b \\
&                   && x \succeq 0 \\
&                   && x \in \mathbb{R}^n
\end{align}
$$


In [ ]:
#Datos
#----------

#INPUT DATA
#----------

c=[2500; 4000; 0; 0; 0] 
b=[200; 100; 750]
A=[1 0 1 0 0; 0 1 0 1 0; 3 5 0 0 1]
m, n = size(A) # m = filas n = columnas

In [ ]:
modelo = Model(solver = CplexSolver())
@variable(modelo, x[1:n] >= 0) # Models x >=0
for i in 1:m # filas
    @constraint(modelo, sum(A[i, j]*x[j] for j in 1:n) == b[i]) # i-ésima restricción 
end 
@objective(modelo,Max,sum(c[i]*x[i] for i in 1:n))
status = solve(modelo) # solves the model  
println(getobjectivevalue(modelo))

# Un modelo algo más complejo

Obviamente, el modelo anterior no tiene mayor dificultad. Alternativamente podríamos intentar resolver algún modelo con algo más de historia (como por ejemplo el Uncapacitated Lot Sizing Problem).

\begin{align}
\text{minimize} \qquad & \sum_{t\in T} c_s s_t+ \sum_{t\in T} c_K y_t \\
 \text{subject to} \quad \quad & s_{t-1} + x_t = s_t + d_t \\
 \qquad \qquad & x_t \leq M y_t \\
 \qquad \qquad & s_0 = 0 \\
 \qquad \qquad & x_t, s_t \in \mathbb{R}^+ \forall t\in T\\
 \qquad \qquad & y_t \in \{0,1\}
\end{align}

$c_s$ y $c_K$ son los costes de inventario (unidad por periodo) y de realizar un pedido en un periodo y $d_t$ la demanda de cada periodo. El conjunto de periodos es $T$. $x_t$ es la cantidad que se recibe en el periodo $t$, mientras que $y_t$ equivale a si se realiza un pedido en el periodo $t$ o no y $s_t$ es el inventario al final del periodo $t$.

El objetivo minimiza la suma de costos de inventario y pedido (el costo por unidad no se tiene en cuenta porque la solución óptima siempre compra exactamente la demanda, $\sum_{t\in T} d_t$.

La primera restricción asegura el cumplimiento de la demanda de cada periodo. La segunda restricción asegura que sólo se compran unidades si se ha realizado un pedido ($M$ es un valor lo suficientemente grande, en este caso basta con $\sum_{t\in T} d_t$). La tercera restricción inicializa el inventario y el resto de restricciones definen el problema.

In [ ]:
T=10
d=[5; 10; 7; 11; 13; 14; 2; 8; 17; 21]
cK=50
cs=2
M=sum(d)

In [ ]:
modelo=Model(solver = CplexSolver())
@variable(modelo, x[1:T]>= 0)
@variable(modelo, y[1:T],Bin)
@variable(modelo, s[0:T]>=0)

#función objetivo a minimizar
@objective(modelo, Min, sum(cs*s[t] for t in 1:T)+sum(cK*y[t] for t in 1:T))

for t in 1:T
    @constraint(modelo,s[t-1]+x[t]==d[t]+s[t])
end
for t in 1:T
    @constraint(modelo,x[t]<=M*y[t])
end
@constraint(modelo,s[0]==0)

#escribimos el modelo
print(modelo)

#resolvemos
status = solve(modelo)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(modelo))
println("**** x = ", getvalue(x))
println("**** y = ", getvalue(y))
println("**** s = ", getvalue(s))

# Set Covering Problem

Veamos un ejemplo un poco más complejo en que es importante considerar que la mayoría de la matriz A está formada por ceros.

El modelo es:

\begin{align}
\text{minimize} \qquad & \sum_{j=1}^{n} c_j x_j \\
 \text{subject to} \quad \quad & \sum_{i=1}^{m} a_{ij} x_j \geq 1 \forall i\in 1,\cdots,m \\
 \qquad \qquad & x_j \in \{0,1\} \forall j\in 1,\cdots,n
\end{align}

Primero vamos a bajar un archivo de datos del mismo 

In [ ]:
run(`wget -O scp41.txt http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/scp41.txt`)

In [ ]:
run(`cat scp41.txt`)

La estructura de datos es algo confusa. Primero tenemos una línea con el número de restricciones (200) y en número de variables (1000). Los siguientes 1000 valores corresponden a los costos de las variables en la función objetivo y después hay la descripción de las 200 restricciones con esta forma (copio la primera)

 17 
 91 214 230 289 351 416 488 491 518 567 720 721 
 735 753 768 928 990 
 
 Que significa que hay 17 no ceros y que las variables 91, ..., 990 adoptan valor en la restriccion.
 
 Como la estructura es un tanto oscura, vamos a tener que crear nuestro propio lector de instancias:

In [ ]:
function readFile(filename)
    f=open(filename,"r")
    s=readstring(f)
    close(f)
    s=replace(s,"\n","")
    s=split(s," ")
    nC=parse(Int64,s[2]) #number of constraints
    nV=parse(Int64,s[3]) #number of variables
    numCoefFo=0 #contadores para procesar la información
    numCoef=0
    numRow=0
    c= Float64[] #coeficientes de la función objetivo
    #método para almacenar una matriz (o un grafo) sparse
    I=Int64[] #puntero a fila
    J=Int64[] # puntero a columna
    V=Float64[] # puntero a valor
    for i in 4:length(s)
        if s[i]!= ""
            if numCoefFo < nV
                push!(c,parse(Float64,s[i]))
                numCoefFo += 1
            else
                if numCoef == 0
                    numRow += 1
                    numCoef = parse(Int64,s[i])
                else
                    numCoef -= 1
                    push!(I,numRow)
                    push!(J,parse(Int64,s[i]))
                    push!(V,1.0)
                end
            end
        end
    end
    A=sparse(I,J,V)
    return nC,nV,c,A #,c,A
end

In [ ]:
constraints,variables,c,A = readFile("scp41.txt")

println("\n\n",constraints," ",variables)
println("\n\n",c)
println("\n\n",A)

In [ ]:
scp=Model(solver = CplexSolver())
@variable(scp, x[1:variables],Bin)

#función objetivo a minimizar
@objective(scp, Min, sum(c[j]*x[j] for j in 1:variables))

for i in 1:constraints
    @constraint(scp,sum(A[i,j]*x[j] for j in 1:variables) >=1 )
end

#escribimos el modelo
print(scp)

#resolvemos
status = solve(scp)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(scp))
println("**** x = ", getvalue(x))



Otra alternativa (que en muchas ocasiones es más práctica) es construir la expresión asociada a cada restricción:

In [ ]:
scpAlt=Model(solver = CplexSolver())
@variable(scpAlt, xAlt[1:variables],Bin)

#función objetivo a minimizar
Expr = AffExpr()
for j in 1:variables
    push!(Expr, c[j], xAlt[j])
end
@objective(scpAlt, Min, Expr)

for i in 1:constraints
    Expr = AffExpr()
    for j in 1:variables
        push!(Expr,A[i,j],xAlt[j])
    end
    @constraint(scpAlt,Expr >= 1)
end
#for i in 1:constraints
#    @constraint(scp,sum(A[i,j]*x[j] for j in 1:variables) >=1 )
#end

#escribimos el modelo
#print(scpAlt)

#resolvemos
status = solve(scpAlt)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(scpAlt))
#println("**** x = ", getvalue(xAlt))
for i in 1:variables
    if getvalue(xAlt[i])>0.99
        println(i,"\t",xAlt[i])
    end
end



# Modificar el comportamiento del solver

Antes de dar introducirnos en otros elementos más complejos, convendría ver cómo eliminar esos logs tan largos que ofrece Cplex (entre otros solvers).

La idea es pasar un "flag" al solver (aquí hay una lista de los parámetros de cplex: https://www.ibm.com/support/knowledgecenter/SSSA5P_12.6.1/ilog.odms.cplex.help/CPLEX/Parameters/topics/introListAlpha.html)


In [ ]:
#creamos un modelo
m = Model(solver = CplexSolver(CPX_PARAM_SCRIND=0))

#y sus variables
@variable(m, x >= 0 )
@variable(m, y >= 0 )

#función objetivo a minimizar
@objective(m, Min, x + y )

#restricción
@constraint(m, x + y >= 1.0 )

#escribimos el modelo
print(m)

#resolvemos
status = solve(m)

#mostramos resultados
println("Status: ",status," Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))